<a href="https://colab.research.google.com/github/Andres8bit/parallel-computing/blob/main/merge_sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-g3iv36nh
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-g3iv36nh
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=7b9064c354cf5b2e111f783f2695b993c8a58d8950da70ae6516a76f13721fce
  Stored in directory: /tmp/pip-ephem-wheel-cache-hkgr0tt_/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include <iostream>


__device__ int co_rank(int k, float* a,int m,float* b,int n);

__device__ void sequential_merge(float* a, int m, float* b, int n, float* c);

__global__ void merge_basic_kernel(float* a,int m,float* b,int n, float* c);

__device__ int co_rank(int k, float* a,int m,float* b,int n);

void print_array(float *array, int width);

int main(){
    float host_a[] = {-25,1,6,9,11,25,46,79,250};
    float host_b[] = {13,22,23,24,55,57,86,99};
    float host_c[] ={0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0};

    float* device_a; float* device_b; float* device_c;

    cudaMalloc((void **)&device_a,9 * sizeof(float));
    cudaMalloc((void **)&device_b,8 * sizeof(float));
    cudaMalloc((void **)&device_c,17 * sizeof(float));
    
    cudaMemcpy(device_a,host_a,9 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_b,host_b,8 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_c,host_c,17 * sizeof(float),cudaMemcpyHostToDevice);

    dim3 dim_grid(1);
    dim3 dim_block(17);

    merge_basic_kernel<<<dim_grid,dim_block>>>(device_a,9,device_b,8,device_c);

    cudaMemcpy(host_c,device_c, 17 * sizeof(float),cudaMemcpyDeviceToHost);

    printf("a: ");
    print_array(host_a,9);

    printf("\nb: ");
    print_array(host_b,8);
    
    printf("\nsorted: ");
    print_array(host_c,17);
    
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    
    return 0;
}


void print_array(float *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");
}

__device__ int co_rank(int k, float* a,int m,float* b,int n){
    int i = k < m ? k : m;
    int j = k - i;
    int i_low = 0 > (k-n) ? 0 : k-n;
    int j_low = 0 > (k-m) ? 0 : k-m;
    int delta = 0;
    bool active = true;

    while(active){
        if ( i > 0 && j < n && a[i-1] > b[j]){
             delta = ((i - i_low + 1) >>1);
             j_low = j;
             j = j + delta;
             i = i - delta;
        }else{
            if ( j > 0 && i < m && b[j-1] >= a[i]){
                 delta = ((j - j_low + 1) >> 1);
                 i_low = i;
                 i = i + delta;
                 j = j - delta;
            }else{
                active = false;
            }
        }
    }
    return i;
}

__device__ void sequential_merge(float* a, int m, float* b, int n, float* c){
    int i = 0;
    int j = 0;
    int k = 0;

    while( i < m && j < n){
         if (a[i] <= b[j])
            c[k++] = a[i++];
        else
            c[k++] = b[j++]; 
    }
    if (i == m)
      for (; j < n; j++)
            c[k++] = b[j];
    else
        for (; i < m;i++)
              c[k++] = a[j];
}

__global__ void merge_basic_kernel(float* a,int m,float* b,int n, float* c){
    int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    int ceil_val = (int)((m + n) / (blockDim.x * gridDim.x)) == (m + n) / (blockDim.x * gridDim.x) ?
                (int)((m + n) / (blockDim.x * gridDim.x)) 
                :(int)((m + n) / (blockDim.x * gridDim.x)) + 1;

    int k_cur =  thread_id * ceil_val;
    int k_next =  ((thread_id + 1)* ceil_val) < m + n ? 
                  ((thread_id + 1) * ceil_val) : m + n;

    int i_cur = co_rank(k_cur,a,m,b,n);
    int i_next = co_rank(k_next,a,m,b,n);
    int j_cur = k_cur - i_cur;
    int j_next = k_next - i_next;

    sequential_merge(&a[i_cur], i_next - i_cur ,&b[j_cur], j_next - j_cur, &c[k_cur]); 
}

a: -25	1	6	9	11	25	46	79	250	

b: 13	22	23	24	55	57	86	99	

sorted: -25	1	6	9	11	13	22	23	24	25	46	55	57	79	86	99	250	



In [5]:
%%cu
#include <iostream>
#include <stdio.h>
#include <math.h>

__global__ void merge_tiled_kernel(int* a,int m,int* b,int n,int* c,int tile_size);

__device__ void sequential_merge(int* a, int m, int* b, int n, int* c);

__device__ int co_rank(int k, int* a,int m,int* b,int n);

void print_array(int *array, int width);

int main(){
    int m = 33000;
    int n = 31000;
    int host_a[m];
    int host_b[n];
    int host_c[m + m];

    int start = rand() % 5000;
    int interval = rand() % 2500;
    host_a[0] = start;
    for (int i = 1 ; i < m ; i++)
        host_a[i] =  host_a[i-1] + interval;
     
    start += rand() % 3000;
    interval = rand() % 2500;
    host_b[0] = start;
    for (int i = 1 ; i < n ; i++)
        host_b[i] =  host_b[i-1] + interval;
    
    for (int i = 0 ; i <  m + n; i++)
        host_b[i] = 0;
    
    int* device_a; int* device_b; int* device_c;

    cudaMalloc((void **)&device_a,m * sizeof(int));
    cudaMalloc((void **)&device_b,n * sizeof(int));
    cudaMalloc((void **)&device_c,(m + n) * sizeof(int));
    
    cudaMemcpy(device_a,host_a,m * sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(device_b,host_b,n * sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(device_c,host_c,(m + n) * sizeof(int),cudaMemcpyHostToDevice);

    dim3 dim_grid(32);
    dim3 dim_block(128);

    merge_tiled_kernel<<<dim_grid,dim_block,(2*1024)*sizeof(int)>>>(device_a,n,device_b,m,device_c,1024);
    merge_tiled_kernel<<<dim_grid,dim_block,(2*1024)*sizeof(int)>>>(device_a,n,device_b,m,device_c,1024);

    cudaMemcpy(host_c,device_c, (m + n) * sizeof(int),cudaMemcpyDeviceToHost);

    printf("\nsorted: ");
    print_array(host_c,m + n);
    
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    
    return 0;
}


void print_array(int *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");
}
__device__ int co_rank(int k, int* a,int m,int* b,int n){
    int i = k < m ? k : m;
    int j = k - i;
    int i_low = 0 > (k-n) ? 0 : k-n;
    int j_low = 0 > (k-m) ? 0 : k-m;
    int delta = 0;
    bool active = true;

    while(active){
        if ( i > 0 && j < n && a[i-1] > b[j]){
             delta = ((i - i_low + 1) >>1);
             j_low = j;
             j = j + delta;
             i = i - delta;
        }else{
            if ( j > 0 && i < m && b[j-1] >= a[i]){
                 delta = ((j - j_low + 1) >> 1);
                 i_low = i;
                 i = i + delta;
                 j = j - delta;
            }else{
                active = false;
            }
        }
    }
    return i;
}

__device__ void sequential_merge(int* a, int m, int* b, int n, int* c){
    int i = 0;
    int j = 0;
    int k = 0;

    while( i < m && j < n){
         if (a[i] <= b[j])
            c[k++] = a[i++];
        else
            c[k++] = b[j++]; 
    }
    if (i == m)
      for (; j < n; j++)
            c[k++] = b[j];
    else
        for (; i < m;i++)
              c[k++] = a[j];
}

__global__ void merge_tiled_kernel(int* a,int m,int* b,int n,int* c,int tile_size)
{
    extern __shared__ int share_ab[];
  
    int* a_share = share_ab;
    int* b_share = &share_ab[tile_size];
    int ceil_val = (int)((m+n)/gridDim.x) == (m+n)/gridDim.x ? 
                             (int)((m+n)/gridDim.x): (int)((m+n)/gridDim.x) + 1;
    int C_cur = blockIdx.x * ceil_val;
    int C_next = ((blockIdx.x + 1) * ceil_val) < (m + n) ? blockIdx.x * ceil_val:m + n; 


    

    if (threadIdx.x == 0.0){    
        a_share[0] = co_rank(C_cur,a,m,b,n);
        a_share[1] = co_rank(C_next,a,m,b,n);
    }
 
    __syncthreads();
    int A_cur = a_share[0];
    int A_next = a_share[1];
    int B_cur = C_cur - A_cur;
    int B_next = C_next - A_next;

 
    __syncthreads();
    int counter = 0;
    int c_length = C_next - C_cur;

    int a_length = A_next - A_cur;
    int b_length = B_next - B_cur;
    int c_completed = 0;
    int a_consumed = 0;
    int b_consumed = 0;
    int total_itrs =  (int)(c_length/tile_size) == c_length/tile_size ? 
                      (int)(c_length/tile_size): (int)(c_length/tile_size) + 1;

 
    int a_min = 0;
    int b_min = 0;
    int c_cur = 0;
    int c_next = 0;
    int a_cur = 0;
    int a_next = 0;
    int b_next = 0;
    int b_cur = 0;
 
    while (counter < total_itrs){
        for (int i = 0; i < tile_size; i += blockDim.x)
          if ( i + threadIdx.x < a_length - a_consumed)
            a_share[i + threadIdx.x] = a[A_cur + a_consumed + i + threadIdx.x];

        for ( int i = 0; i < tile_size;i += blockDim.x)
          if ( i + threadIdx.x < b_length - b_consumed)
            b_share[i + threadIdx.x] = b[B_cur + b_consumed + i + threadIdx.x];

    __syncthreads();

    c_cur = threadIdx.x * (tile_size/blockDim.x);
    c_next = (threadIdx.x + 1) * (tile_size/blockDim.x);
    c_cur = (c_cur <= c_length - c_completed) ? c_cur : c_length - c_completed;
    c_next = (c_next <= c_length - c_completed) ? c_next : c_length - c_completed;

    a_min = tile_size < (a_length - a_consumed) ? tile_size : (a_length - a_consumed);
    b_min = tile_size < (b_length - b_consumed) ? tile_size : (b_length - b_consumed);

    a_cur = co_rank(c_cur,a_share,a_min,b_share,b_min);
    b_cur = c_cur - a_cur;

    a_next = co_rank(c_next,a_share,a_min,b_share,b_min);
    b_next = c_next - a_next;

    sequential_merge(a_share + a_cur, a_next - a_cur,b_share + b_cur,
                     b_next - b_cur, c + C_cur + c_completed + c_cur);
    
    counter++;
    c_completed += tile_size;
    a_consumed += co_rank(tile_size, a_share, tile_size, b_share,tile_size);
    b_consumed = c_completed - a_consumed;
    __syncthreads();
    }
}


sorted: 0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0